In [2]:
from datasets import load_dataset

# Load a small slice first to test
ds = load_dataset("wikimedia/wikipedia", "20231101.en", split="train[:1%]")

print(ds[0])


s:\ML-project\GEN-AI-H2S-notebooks-\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'id': '12', 'url': 'https://en.wikipedia.org/wiki/Anarchism', 'title': 'Anarchism', 'text': 'Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. As a historically left-wing movement, this reading of anarchism is placed on the farthest left of the political spectrum, usually described as the libertarian wing of the socialist movement (libertarian socialism).\n\nHumans have lived in societies without formal hierarchies long before the establishment of states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist ideas are found all throughout history, modern anarchism emerged from the Enlightenment. During

In [3]:
import json

fever_unified = []

with open("train.jsonl", "r") as f:
    for line in f:
        item = json.loads(line)
        if item['label'] != "SUPPORTS":
            continue  # skip REFUTES / NOT ENOUGH INFO

        for eg_idx, evidence_group in enumerate(item['evidence']):
            for ev_idx, ev in enumerate(evidence_group):
                article_title = ev[2] if ev[2] is not None else "NA"
                passage_id = f"fever_{item['id']}_{eg_idx}_{ev_idx}"
                entry = {
                    "id": passage_id,
                    "url": f"https://en.wikipedia.org/wiki/{article_title}" if article_title != "NA" else "NA",
                    "title": article_title,
                    "text": item['claim']  # simplest: use claim as text; later can replace with actual evidence text if available
                }
                fever_unified.append(entry)

print(f"Total SUPPORTS entries: {len(fever_unified)}")


FileNotFoundError: [Errno 2] No such file or directory: 'train.jsonl'

In [ ]:
fever_unified

[{'id': 'fever_75397_0_0',
  'url': 'https://en.wikipedia.org/wiki/Nikolaj_Coster-Waldau',
  'title': 'Nikolaj_Coster-Waldau',
  'text': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.'},
 {'id': 'fever_75397_0_1',
  'url': 'https://en.wikipedia.org/wiki/Fox_Broadcasting_Company',
  'title': 'Fox_Broadcasting_Company',
  'text': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.'},
 {'id': 'fever_150448_0_0',
  'url': 'https://en.wikipedia.org/wiki/Roman_Atwood',
  'title': 'Roman_Atwood',
  'text': 'Roman Atwood is a content creator.'},
 {'id': 'fever_150448_1_0',
  'url': 'https://en.wikipedia.org/wiki/Roman_Atwood',
  'title': 'Roman_Atwood',
  'text': 'Roman Atwood is a content creator.'},
 {'id': 'fever_214861_0_0',
  'url': 'https://en.wikipedia.org/wiki/History_of_art',
  'title': 'History_of_art',
  'text': 'History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, film, photography and graphic

In [ ]:
import pandas as pd
fev_ds = pd.DataFrame(fever_unified)

In [ ]:
ds_wiki = ds.to_pandas()

In [ ]:
ds_wiki.head()

,id,url,title,text
0,12,https://en.wikipedia.org/wiki/Anarchism,Anarchism,Anarchism is a political philosophy and moveme...
1,39,https://en.wikipedia.org/wiki/Albedo,Albedo,Albedo (; ) is the fraction of sunlight that i...
2,290,https://en.wikipedia.org/wiki/A,A,"A, or a, is the first letter and the first vow..."
3,303,https://en.wikipedia.org/wiki/Alabama,Alabama,Alabama () is a state in the Southeastern regi...
4,305,https://en.wikipedia.org/wiki/Achilles,Achilles,"In Greek mythology, Achilles ( ) or Achilleus ..."


In [ ]:
df_ret = pd.concat([ds_wiki, fev_ds])

In [ ]:
df_ret

,id,url,title,text
0,12,https://en.wikipedia.org/wiki/Anarchism,Anarchism,Anarchism is a political philosophy and moveme...
1,39,https://en.wikipedia.org/wiki/Albedo,Albedo,Albedo (; ) is the fraction of sunlight that i...
2,290,https://en.wikipedia.org/wiki/A,A,"A, or a, is the first letter and the first vow..."
3,303,https://en.wikipedia.org/wiki/Alabama,Alabama,Alabama () is a state in the Southeastern regi...
4,305,https://en.wikipedia.org/wiki/Achilles,Achilles,"In Greek mythology, Achilles ( ) or Achilleus ..."
...,...,...,...,...
193751,fever_13114_0_0,https://en.wikipedia.org/wiki/Gimli_-LRB-Middl...,Gimli_-LRB-Middle-earth-RRB-,J. R. R. Tolkien created Gimli.
193752,fever_13114_1_0,https://en.wikipedia.org/wiki/Gimli_-LRB-Middl...,Gimli_-LRB-Middle-earth-RRB-,J. R. R. Tolkien created Gimli.
193753,fever_152180_0_0,https://en.wikipedia.org/wiki/Susan_Sarandon,Susan_Sarandon,Susan Sarandon is an award winner.
193754,fever_152180_1_0,https://en.wikipedia.org/wiki/Susan_Sarandon,Susan_Sarandon,Susan Sarandon is an award winner.


In [ ]:
df_ret_texts_list = df_ret['text'].to_list()

In [ ]:
type(df_ret_texts_list)

NameError: name 'df_ret_texts_list' is not defined

In [2]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()
API_key = os.getenv("GOOGLE_FACT_CHECK_API")


url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

params = {
    'key': API_key,
    'query': 'covid vaccine', 
    'languageCode': 'en-US',
    'pageSize': 100
}

response = requests.get(url,params=params)
len(response.json()['claims'])

100

In [3]:
response.json()['claims'][0]
i=0
for claim in response.json()['claims']:
    # print(claim.get('claimReview')[0].get('title',''))
    print(claim
          )
    i+=1
    if i==4:
        break



{'text': 'The Covid vaccine is unsafe.', 'claimReview': [{'publisher': {'name': 'Full Fact', 'site': 'fullfact.org'}, 'url': 'https://fullfact.org/health/aseem-malhotra-reform-conference-vaccines/', 'title': 'Covid vaccine claims at the Reform UK conference: fact checked – Full Fact', 'reviewDate': '2025-09-08T00:00:00Z', 'textualRating': 'This is not correct, according to regulators and other experts. The Covid vaccines approved for use in the UK have met strict standards for safety and effectiveness.', 'languageCode': 'en'}]}
{'text': 'A leading oncologist thinks it’s highly likely that the Covid vaccines have been a significant factor in the cancer of members of the Royal Family.', 'claimReview': [{'publisher': {'name': 'Full Fact', 'site': 'fullfact.org'}, 'url': 'https://fullfact.org/health/aseem-malhotra-reform-conference-vaccines/', 'title': 'Covid vaccine claims at the Reform UK conference: fact checked – Full Fact', 'reviewDate': '2025-09-08T00:00:00Z', 'textualRating': 'There

In [4]:
def fetch_google_facts(query,num_iter = 1, pages = 100):
    
    load_dotenv()
    API_key_google = os.getenv("GOOGLE_FACT_CHECK_API")


    url_google = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

    ds=[]
    next_page_token = None
    for _ in range(num_iter):

        params = {
        'key': API_key_google,
        'query': query, 
        'languageCode': 'en-US',
        'pageSize': pages
        }

        if next_page_token:
            params['pageToken'] = next_page_token

        try:
            response = requests.get(url_google,params=params)
            response.raise_for_status()

            for claim in response.json()['claims']:
                ds.append(
                    {'title':claim.get('claimReview',[])[0].get('title',''),
                     'text':claim.get('text',''),
                     'url':claim.get('claimReview',[])[0].get('url',''), 
                     'reviewDate':claim.get('claimReview')[0].get('reviewDate',''),
                     })
                
            next_page_token = response.json().get('nextPageToken')
            if not next_page_token:
                break

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            break
        except Exception as e:
            print(f"Error processing response: {e}")
            break

    return ds


In [5]:
ds1 = fetch_google_facts('indian government')
ds1,len(ds1)

([{'title': 'Did Govt Declare Sept 3 to 6, 2025, as Public Holidays? Fact Check – DigitEye India',
   'text': 'The Indian Government has declared September 3-6 2025 as public holidays.',
   'url': 'https://digiteye.in/did-govt-declare-sept-3-to-6-2025-as-public-holidays-fact-check/',
   'reviewDate': '2025-09-05T00:00:00Z'},
  {'title': 'Viral ₹1 Lakh Rupee Coin Is Not Issued by the Government of India',
   'text': 'Viral video shows a genuine one lakh rupee coin issued by the Indian government.',
   'url': 'https://factly.in/viral-%E2%82%B91-lakh-rupee-coin-is-not-issued-by-the-government-of-india/',
   'reviewDate': '2025-09-05T00:00:00Z'},
  {'title': 'Fact Check: Has Indian Govt Promised To Bring Back A Truck Driver From US In 24 Hours? Here’s the Truth',
   'text': 'Indian Govt Promised To Bring Back Harjinder Singh From US In 24 Hours',
   'url': 'https://www.newsmobile.in/nm-fact-checker/fact-check-indian-govt-has-not-promised-to-bring-back-harjinder-singh-from-us-in-24-hours/',

In [6]:
ds2 = fetch_google_facts('indian politics')
len(ds2)

9

In [8]:
ds = ds1 + ds2
len(ds)

109

In [9]:
import pandas as pd
df = pd.DataFrame(ds)


In [10]:

url_news_api = "https://newsapi.org/v2/everything"

load_dotenv()
api_key_news = os.getenv('NEWS_API')

sort_by  =['relevancy', 'popularity', 'publishedAt']
params = {
    "apiKey":api_key_news,
    "q":'indian actors',
    "sortBY":sort_by[0],
    "pageSize":69

}

res = requests.get(url_news_api,params=params)
res.json().get('articles')

[{'source': {'id': None, 'name': 'The Indian Express'},
  'author': 'Entertainment Desk',
  'title': 'Actor, who worked with Amitabh Bachchan and Sridevi, was not at peace with fame and left the industry at the peak of his career',
  'description': 'Mangal Dhillon was one of the most prominent television actors of his time, known for iconic serials like Buniyaad, Junoon, and Noorjahan. At the peak of his career, he was also among the highest-paid actors on Indian television.',
  'url': 'https://indianexpress.com/article/entertainment/bollywood/actor-mangal-dhillon-was-not-at-peace-with-fame-and-left-the-industry-at-the-peak-of-his-career-10190788/',
  'urlToImage': 'https://images.indianexpress.com/2025/08/mangal-01.png',
  'publishedAt': '2025-08-15T08:53:32Z',
  'content': 'Whenever Mangal Dhillon appeared on screen, the first thing audiences noticed was his commanding baritone voice. A prominent figure in Indian television and film during the 1980s and 1990s, Mangal wo… [+4878 chars

In [11]:
arl=res.json().get('articles')
arl[0]
len(arl)
res.json()

{'status': 'ok',
 'totalResults': 256,
 'articles': [{'source': {'id': None, 'name': 'The Indian Express'},
   'author': 'Entertainment Desk',
   'title': 'Actor, who worked with Amitabh Bachchan and Sridevi, was not at peace with fame and left the industry at the peak of his career',
   'description': 'Mangal Dhillon was one of the most prominent television actors of his time, known for iconic serials like Buniyaad, Junoon, and Noorjahan. At the peak of his career, he was also among the highest-paid actors on Indian television.',
   'url': 'https://indianexpress.com/article/entertainment/bollywood/actor-mangal-dhillon-was-not-at-peace-with-fame-and-left-the-industry-at-the-peak-of-his-career-10190788/',
   'urlToImage': 'https://images.indianexpress.com/2025/08/mangal-01.png',
   'publishedAt': '2025-08-15T08:53:32Z',
   'content': 'Whenever Mangal Dhillon appeared on screen, the first thing audiences noticed was his commanding baritone voice. A prominent figure in Indian television a

In [12]:

def fetch_news_org(query:str , page_size:int = 100,num_iter = 12,sort_by_index:int = 0):

    url_news_api = "https://newsapi.org/v2/everything"

    load_dotenv()
    api_key_news = os.getenv('NEWS_API')
    if not api_key_news:
            raise ValueError("NEWS_API environment variable not set")


    sort_by  =['relevancy', 'popularity', 'publishedAt']
    if sort_by_index < 0 or sort_by_index >= len(sort_by):
        sort_by_index = 0

    news_ds = []
    
    for page in range(1,num_iter+1):

        params = {
            "apiKey":api_key_news,
            "q":query,
            "sortBY":sort_by[sort_by_index],
            "pageSize": min(page_size,100),
            'page':page
            }

        try:
            res = requests.get(url_news_api,params=params)

            if res.json()['status']=="ok":
                 print(f'status | {res.json()['status']}')
            

            data = res.json().get('articles')
            if not data:
                 print(f'Could not find any article at page {page}')
                 break
   

            for article in data:
                if not article.get('content'):
                     continue
 
                news_ds.append({
                    "title":article.get('title',''),
                    "text":article.get('content',""),
                    "url":article.get('url',''),
                    "url_image":article.get('urlToImage',""),
                    "Published_Date":article.get('publishedAt','')

                })

            if len(news_ds)>=res.json().get('totalResults',0):
                 print(f"No More Results | reached {res.json().get('totalResults',0)} Results")
                 break
            
        except requests.exceptions.RequestException as e:
            print(f"Request error on iteration {i+1}: {e}")
            break
        except Exception as e:
            print(f"Unexpected error on iteration {i+1}: {e}")
            break
    
    print(f"Fetched {len(news_ds)} news articles")
    return news_ds




In [13]:
news_db = fetch_news_org('conjuring',500,100,1)

status | ok
Could not find any article at page 2
Fetched 100 news articles


In [15]:
news_db[0]

{'title': 'Showrunner wants to turn you into a happy little content prompter for the ‘Netflix of AI’',
 'text': '<ul><li></li><li></li><li></li></ul>\r\nShowrunner wants to turn you into a happy little content prompter for the Netflix of AI\r\nFable founder Edward Saatchi aims to gamify Hollywoods pivot to AI one p… [+12829 chars]',
 'url': 'https://www.theverge.com/ai-artificial-intelligence/762594/fable-showrunner-edwatch-saatchi-interview',
 'url_image': 'https://platform.theverge.com/wp-content/uploads/sites/2/2025/08/Exit-Valley.jpeg?quality=90&strip=all&crop=2.059%2C0%2C95.882%2C100&w=1200',
 'Published_Date': '2025-08-29T20:10:07Z'}

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.ndtv.com/topics",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
}



url="https://archives.ndtv.com/articles/2025-01.html"

response  = requests.get(url,headers=headers)
response.status_code

403

In [24]:
def get_ndtv_rss_feeds():
    """
    Get news from NDTV's RSS feeds
    """
    rss_feeds = {
        'top_stories': 'https://feeds.feedburner.com/ndtvnews-top-stories',
        'india': 'https://feeds.feedburner.com/ndtvnews-india-news',
        'world': 'https://feeds.feedburner.com/ndtvnews-world-news',
        'sports': 'https://feeds.feedburner.com/ndtvnews-sports',
        'entertainment': 'https://feeds.feedburner.com/ndtvnews-entertainment',
        'business': 'https://feeds.feedburner.com/ndtvnews-business'
    }
    
    all_articles = []
    
    for category, url in rss_feeds.items():
        try:
            print(f"Fetching RSS feed: {category}")
            response = requests.get(url, timeout=10)
            
            if response.status_code != 200:
                print(f"Failed to fetch RSS feed: {response.status_code}")
                continue
                
            soup = BeautifulSoup(response.text, 'html.parser')
            items = soup.find_all('item')
            
            for item in items:
                title = item.find('title')
                link = item.find('link')
                description = item.find('description')
                pub_date = item.find('pubDate')
                
                if title and link:
                    all_articles.append({
                        'title': title.get_text().strip(),
                        'url': link.get_text().strip(),
                        'description': description.get_text().strip() if description else '',
                        'published_date': pub_date.get_text().strip() if pub_date else '',
                        'category': category,
                        'source': 'NDTV RSS'
                    })
            
            
            
        except Exception as e:
            print(f"Error processing RSS feed {category}: {e}")
    
    print(f"Found {len(all_articles)} articles from RSS feeds")
    return all_articles

# Get articles from RSS
rss_articles = get_ndtv_rss_feeds()
rss_df = pd.DataFrame(rss_articles)

Fetching RSS feed: top_stories
Fetching RSS feed: india
Fetching RSS feed: world
Fetching RSS feed: sports
Fetching RSS feed: entertainment
Fetching RSS feed: business
Found 60 articles from RSS feeds


In [52]:
rss_df.iloc[0]

title             Court Relief For Sonia Gandhi Over 'Name In Vo...
url                                                                
description       A Delhi court has dismissed a petition seeking...
published_date                                                     
category                                                top_stories
source                                                     NDTV RSS
Name: 0, dtype: object